In [ ]:
!nvidia-smi

Tue Nov 29 08:26:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -q torchmetrics
!pip install -q transformers==4.20.1
!pip install -q sentencepiece
!pip install -q kaggle --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     |████████████████▎               | 834.1 MB 1.3 MB/s eta 0:10:10tcmalloc: large alloc 1147494400 bytes == 0x665e4000 @  0x7fc51a829615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████▋           | 1055.7 MB 1.3 MB/s eta 0:07:32tcmalloc: large alloc 1434370048 bytes == 0x3e0c000 @  0x7fc51a829615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |██████████████████████████▏     | 1

In [ ]:
prepare_data = True
import os
if os.path.exists('/content/data'):
    prepare_data = False

In [ ]:
if prepare_data:    
    from google.colab import files, drive

    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))
    
    # Then move kaggle.json into the folder where the API expects to find it.
    !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

    !mkdir data && cd data && kaggle competitions download -c feedback-prize-english-language-learning
    !unzip /content/data/feedback-prize-english-language-learning.zip -d /content/data/

    # !mkdir data/pretrained && cd data/pretrained && kaggle datasets download -d sunpnwt12/fb3-pretrained-s42
    # !unzip /content/data/pretrained/fb3-pretrained-s42.zip -d /content/data/pretrained/

    drive.mount('/content/drive')

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes
100% 2.80M/2.80M [00:00<00:00, 5.52MB/s]
100% 2.80M/2.80M [00:00<00:00, 4.71MB/s]
Archive:  /content/data/feedback-prize-english-language-learning.zip
  inflating: /content/data/sample_submission.csv  
  inflating: /content/data/test.csv  
  inflating: /content/data/train.csv  
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import sys
import datetime
import itertools
import gc; gc.enable()
import random
import warnings
import yaml
import shutil
import types
from pathlib import Path
from tabulate import tabulate
from tqdm.notebook import trange, tqdm
warnings.filterwarnings("ignore")

os.system('pip install -q iterative-stratification==0.1.7')
import iterstrat
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
print(f'iterstart version: {iterstrat.__version__}')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler

from torchmetrics.functional import mean_squared_error

# import optuna

# sys.path.append('/kaggle/input/attrdict/')
# from attrdict import AttrDict

# os.system('pip install --root-user-action=ignore --force-reinstall transformers==4.22.1')
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig, DataCollatorWithPadding
print(f'transfromers version: {transformers.__version__}')

os.environ["TOKENIZERS_PARALLELISM"] = "false"

iterstart version: 0.1.6
transfromers version: 4.20.1


# Config

In [ ]:
def get_model_path_dict(source, fold_num_list):
    model_path_dict = {
        'yaml': f'{source}/config.yml',
        'config': [cf for cf in Path(source).glob('*_config.pt')][0].as_posix(),
        'tokenizer': f'{source}/tokenizers/',
        'models': [[f_n, f'{source}/best-epoch-fold{f_n}.pt'] for f_n in fold_num_list],
        'tables': {
            'train_result': f'{source}/train_result.csv',
            'best_result': f'{source}/best_result.csv',
            'cv_result': f'{source}/cv_result.csv',
        },
        'log': f'{source}/log.txt'
    }
    return model_path_dict

# Seeding

In [ ]:
#https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964
def seed_everything(seed: int):    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
# seed_everything(INFERCONF.seed)


# Dataset

In [ ]:
data_path = '/content/data'

TRAIN_PATH = f'{data_path}/train.csv'
TEST_PATH = f'{data_path}/test.csv'
SAMP_SUB = f'{data_path}/sample_submission.csv'

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
samp_sup = pd.read_csv(SAMP_SUB)

In [ ]:
def split_cv(conf, df_):
    df = df_.copy(deep=True)
    mskf = MultilabelStratifiedKFold(n_splits=conf.num_folds, shuffle=True, random_state=conf.seed)
    y = df[conf.target_cols]

    for n, (train_index, valid_index) in enumerate(mskf.split(df, y)):
        df.loc[ｖalid_index, 'fold'] = int(n)
    
    df['fold'] = df['fold'].astype(int)
        
    return df

In [ ]:
class FB3Dataset(Dataset):
    def __init__(self, conf, df, tokenizer, fold):
        self.text_id = df['text_id'].reset_index(drop=True)
        self.labels = df[conf.target_cols].reset_index(drop=True)
        self.full_texts = df['full_text'].reset_index(drop=True)
        self.tokenizer = tokenizer
        if conf.dynamic_max_len:
            self.max_len = self._get_max_len()
        else:
            self.max_len = conf.static_max_len_list[fold]
        
    def __len__(self):
        return len(self.full_texts)
    
    def __getitem__(self, idx):
        text_id = self.text_id.loc[idx]
        token = self._get_token(idx)
        
        return text_id, token
    
    def _get_token(self, idx):
        tokenized = self.tokenizer(
                        self.full_texts.loc[idx],
                        add_special_tokens=True,
                        max_length=self.max_len,
                        pad_to_max_length=True,
                        truncation=True,
                        return_tensors=None,
                )
        return {k: torch.tensor(v, dtype=torch.long) for k, v in tokenized.items()} # stack tensor
    
    # get longest max_len
    # https://www.kaggle.com/code/yasufuminakama/fb3-deberta-v3-base-baseline-train/notebook
    def _get_max_len(self):
        lengths = []
        for text in self.full_texts.fillna('').values:
            length = len(self.tokenizer(text, add_special_tokens=False)['input_ids'])
            lengths.append(length)
        return max(lengths) + 2 # cls & sep

# Model

In [ ]:
# https://www.kaggle.com/code/maunish/clrp-pytorch-roberta-finetune/notebook
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

# https://www.kaggle.com/competitions/feedback-prize-english-language-learning/discussion/361678
class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.LayerNorm(in_dim),
            nn.GELU(),
            nn.Linear(in_dim, 1),
        )

    def forward(self, x, mask):
        w = self.attention(x).float()
        w[mask==0]=float('-inf')
        w = torch.softmax(w,1)
        x = torch.sum(w * x, dim=1)
        return x    
    
# https://www.kaggle.com/code/rhtsingh/utilizing-transformer-representations-efficiently
class HiddenAttentionPooling(nn.Module):
    def __init__(self, num_layers, hidden_size, hiddendim_fc):
        super().__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_fc = hiddendim_fc
        self.dropout = nn.Dropout(0.1)

        q_t = torch.normal(mean=0.0, std=0.02, size=(1, self.hidden_size))
        self.q = nn.Parameter(q_t).float()
        w_ht = torch.normal(mean=0.0, std=0.02, size=(self.hidden_size, self.hiddendim_fc))
        self.w_h = nn.Parameter(w_ht).float()

    def forward(self, all_hidden_states):
        hidden_states = torch.stack([all_hidden_states[layer_i][:, 0].squeeze()
                                     for layer_i in range(1, self.num_hidden_layers+1)], dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)
        out = self.attention(hidden_states)
        out = self.dropout(out)
        return out

    def attention(self, h):
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = F.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        return v

class ConcatPooling(nn.Module):
    def __init__(self, pooling_last=4):
        super().__init__()
        self.pooling_last = pooling_last
        
    def forward(self, all_hidden_states):
        concat_pooling = torch.cat(tuple(all_hidden_states[-l] for l in range(1, self.pooling_last + 1)), -1)
#         concat_pooling = concat_pooling.mean(dim=1) # average instead of select only one
        concat_pooling = concat_pooling[:, 0] # select the first one
        return concat_pooling

# https://www.kaggle.com/competitions/google-quest-challenge/discussion/129840
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_layers=12, init_std=0.02):
        super().__init__()
        self.num_layers = num_layers
        weights_init = torch.zeros(self.num_layers).float()
        weights_init.data[:-1] = -3
        self.layer_weights = torch.nn.Parameter(weights_init)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, all_hidden_states):
        all_layer_encoders = torch.stack(
            [self.dropout(layer) for layer in all_hidden_states[-self.num_layers:]], dim=0
        )
        averaged_layers = (torch.softmax(self.layer_weights, dim=0).unsqueeze(1).unsqueeze(1).unsqueeze(1) * all_layer_encoders).sum(0)
        return averaged_layers
        
class MeanPooling(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class CustomModel(nn.Module):
    def __init__(self, conf, fold_num, config_path=None):
        super().__init__()
        if not config_path:
            self.model_conf = AutoConfig.from_pretrained(conf.model_name, output_hidden_states=True)
            self.model_conf = self._set_dropout(self.model_conf)
            self.backbone = AutoModel.from_pretrained(conf.model_name, config=self.model_conf)
        else:
            self.model_conf = torch.load(config_path)
            self.backbone = AutoModel.from_config(self.model_conf)
        if conf.gradient_checkpointing:
            self.backbone.gradient_checkpointing_enable()
        
        if not config_path:
            for layer in self.backbone.encoder.layer[-conf.reinit_last_layers:]:
                for module in layer.modules():
                    self._init_weights(module)
                    
        self.pooling_strategy = conf.pooling_strategy_list[fold_num]
        if self.pooling_strategy == 'mean_pooling':
            self.pooler = MeanPooling()
            
        elif self.pooling_strategy == 'concat_pooling':
            self.pooler = ConcatPooling(conf.concat_pooling_last)
            
        elif self.pooling_strategy == 'attn_pooling': 
            self.attn_pooler = AttentionPooling(self.model_conf.hidden_size) 
            for attn_module in self.attn_pooler.modules():
                self._init_weights(attn_module)
            
        elif self.pooling_strategy == 'wlp_attn_pooling':
            self.wlp_pooler = WeightedLayerPooling(self.model_conf.num_hidden_layers, self.model_conf.initializer_range)
            self.attn_pooler = AttentionPooling(self.model_conf.hidden_size)
            for attn_module in self.attn_pooler.modules():
                self._init_weights(attn_module)
                
        elif self.pooling_strategy == 'concat_h_attn_mean_pooling':
            self.hattn_pooler = HiddenAttentionPooling(self.model_conf.num_hidden_layers, self.model_conf.hidden_size, self.model_conf.hidden_size)
            self.mean_pooler = MeanPooling()

        elif self.pooling_strategy == 'concat_attn_mean_pooling':
            self.attn_pooler = AttentionPooling(self.model_conf.hidden_size)
            for attn_module in self.attn_pooler.modules():
                self._init_weights(attn_module)
            self.mean_pooler = MeanPooling()
            
        else:
            raise Exception('Invalid pooling strategy')

        if self.pooling_strategy in ['mean_pooling', 'attn_pooling', 'wlp_attn_pooling']:
            hidden_size = self.model_conf.hidden_size
        elif self.pooling_strategy in ['concat_pooling']:
            hidden_size = self.model_conf.hidenn_size * conf.concat_pooling_last
        elif self.pooling_strategy in ['concat_h_attn_mean_pooling', 'concat_attn_mean_pooling']:
            hidden_size = self.model_conf.hidden_size * 2
        else:
            raise Exception('Cannot create fc layer.')
            
        self.multi_dropout = conf.multi_dropout
        if self.multi_dropout:
            self.dropout1 = nn.Dropout(conf.multi_dropout_p[0])
            self.dropout2 = nn.Dropout(conf.multi_dropout_p[1])
            self.dropout3 = nn.Dropout(conf.multi_dropout_p[2])
            self.dropout4 = nn.Dropout(conf.multi_dropout_p[3])
            self.dropout5 = nn.Dropout(conf.multi_dropout_p[4])
        else:
            self.dropout0 = nn.Dropout(0.1)
        
        self.fc = nn.Linear(hidden_size, conf.num_labels)
        if conf.reinit_method is not None:
            self._init_weights2_([self.fc], conf.reinit_method)
        else:
            self._init_weights(self.fc)

        self.use_ln = conf.use_ln
        if self.use_ln:
            self.ln = nn.LayerNorm(hidden_size)
            self._init_weights(self.ln)
        
    def _set_dropout(self, conf, ratio=0.):
        conf.attention_probs_dropout_prob = ratio
        conf.hidden_dropout = ratio 
        conf.hidden_dropout_prob = ratio
        conf.pooler_dropout = ratio
        return conf

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.model_conf.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.model_conf.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def _init_weights2_(self, module_lst, method):
        for module in module_lst:
            for param in module.parameters():
                if param.dim() > 1:
                    if method == 'kaiming_normal':
                        nn.init.kaiming_normal_(param)
                    elif method == 'xavier_normal':
                        nn.init.xavier_normal_(param)
                    elif method == 'orthoganol':
                        nn.init.orthogonal_(param)
                    else:
                        raise Exception('The method is invalid')
        
    def forward(self, inputs):
        backbone_outputs = self.backbone(**inputs)
        if self.pooling_strategy == 'mean_pooling':
            last_hidden_states = backbone_outputs['last_hidden_state']
            pooler_outputs = self.pooler(last_hidden_states, inputs['attention_mask'])
            
        elif self.pooling_strategy == 'concat_pooling':
            all_hidden_states = torch.stack(backbone_outputs['hidden_states'])
            pooler_outputs = self.pooler(all_hidden_states)
            
        elif self.pooling_strategy == 'attn_pooling':
            last_hidden_states = backbone_outputs['last_hidden_state']
            pooler_outputs = self.attn_pooler(last_hidden_states, inputs['attention_mask'])
            
        elif self.pooling_strategy == 'wlp_attn_pooling':
            all_hidden_states = torch.stack(backbone_outputs['hidden_states'])
            wlp_pooler = self.wlp_pooler(all_hidden_states)
            pooler_outputs = self.attn_pooler(wlp_pooler, inputs['attention_mask'])
            
        elif self.pooling_strategy == 'concat_h_attn_mean_pooling':
            last_hidden_states = backbone_outputs['last_hidden_state']
            all_hidden_states = torch.stack(backbone_outputs['hidden_states'])
            hattn_outputs = self.hattn_pooler(all_hidden_states)
            mean_outputs = self.mean_pooler(last_hidden_states, inputs['attention_mask'])
            pooler_outputs = torch.cat((hattn_outputs, mean_outputs), -1)
            
        elif self.pooling_strategy == 'concat_attn_mean_pooling':
            last_hidden_states = backbone_outputs['last_hidden_state']
            attn_outputs = self.attn_pooler(last_hidden_states, inputs['attention_mask'])
            mean_outputs = self.mean_pooler(last_hidden_states, inputs['attention_mask'])
            pooler_outputs = torch.cat((attn_outputs, mean_outputs), -1)
            
        if self.use_ln:
            pooler_outputs = self.ln(pooler_outputs)
            
        if self.multi_dropout:
            x1 = self.fc(self.dropout1(pooler_outputs))
            x2 = self.fc(self.dropout2(pooler_outputs))
            x3 = self.fc(self.dropout3(pooler_outputs))
            x4 = self.fc(self.dropout4(pooler_outputs))
            x5 = self.fc(self.dropout5(pooler_outputs))
            
            outputs = (x1 + x2 + x3 + x4 + x5) / 5

        else:
            outputs = self.fc(self.dropout0(pooler_outputs))
            
        return outputs

# Helper

In [ ]:
# optimize padding size
# https://www.kaggle.com/code/yasufuminakama/fb3-deberta-v3-base-baseline-train/notebook
def get_dataloader(conf, df, tokenizer, fold_num):
    
    valid_dataset = FB3Dataset(conf, df[df['fold'] == fold_num], tokenizer, fold_num)

    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=1,
        num_workers=4,
        shuffle=False,
        pin_memory=True,
        drop_last=False
    )
    
    return valid_dataloader

def get_tokenizer(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def load_model(conf, device, fold_num, config_path, pretrained_model_path):
    model = CustomModel(conf, fold_num, config_path=config_path)
    state_dict = torch.load(pretrained_model_path, map_location=device)['model_state_dict']
    model.load_state_dict(state_dict)
    print(f'Loaded "{pretrained_model_path}"')
    return model

def scores_with_MCRMSE(y_predicteds, y_targets):
    cols_score = []
    col_wise = y_targets.shape[1]
    for col in range(col_wise):
        y_predicted = torch.as_tensor(y_predicteds[:, col])
        y_target = torch.as_tensor(y_targets[:, col])
        rmse = mean_squared_error(y_predicted, y_target, squared=False) # returns RMSE value if set to False
        cols_score.append(rmse)
    score_mean = torch.mean(torch.as_tensor(cols_score))
    return score_mean, cols_score

def get_oof_df(targets, predictions):
    text_id = targets['text_id']
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    oof_df = pd.DataFrame()
    
    oof_df[target_cols] = targets[target_cols].values
    oof_df[[f'pred_{col}' for col in target_cols]] = predictions[target_cols].values
    oof_df = pd.concat([text_id, oof_df], axis=1)
    
    return oof_df

def calculate_oof_cv(oof_df):
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    predictions = oof_df[[f'pred_{col}' for col in target_cols]].values
    targets = oof_df[target_cols].values
    score_mean, cols_score = scores_with_MCRMSE(predictions, targets)
    return score_mean.item(), torch.stack(cols_score)

# Inference

In [ ]:
def inference_fn(model, device, dataloader, tokenizer):
    text_ids_list = []
    predictions = []
    
    model.eval()
    model.to(device)

    for text_ids, inputs in tqdm(dataloader):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        text_ids = text_ids[0]

        with torch.no_grad():
            outputs = model(inputs)
        text_ids_list.append(text_ids)
        predictions.append(outputs)
    
    predictions_stack = torch.stack(predictions)
    
    return text_ids_list, predictions_stack

def inference_cv(model_path_dict, df):
#     shutil.copyfile(model_path_dict['yaml'], Path('./', 'config.yml'))
#     shutil.copyfile(model_path_dict['log'], Path('./', 'log.txt'))
#     for k, v in model_path_dict['tables'].items():
#         shutil.copyfile(v, Path('./', Path(v).name))
    
    with open(model_path_dict['yaml']) as file:
        conf = types.SimpleNamespace(**yaml.safe_load(file))
    
    seed_everything(conf.seed)
    df = split_cv(conf, df)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    text_id_list = []
    fold_predictions_list = []
    
    model_config_path = model_path_dict['config']
    tokenizer = get_tokenizer(model_path_dict['tokenizer'])
    tables_path = model_path_dict['tables']
    
    print(f'Using "{model_config_path}"')
    
    for fold_num, model_path in model_path_dict['models']:
        model = load_model(conf, device, fold_num, model_config_path, model_path)
        valid_dataloader = get_dataloader(conf, df, tokenizer, fold_num) # bs 1 inference
        
        fold_text_id_list, fold_predictions = inference_fn(model, device, valid_dataloader, tokenizer)
        
        text_id_list.append(fold_text_id_list)
        fold_predictions_list.append(fold_predictions)
    
    cv_mean = torch.cat(fold_predictions_list).squeeze().cpu().numpy()
    cv_mean_df = pd.DataFrame(cv_mean, columns=conf.target_cols)
    text_id_df = pd.DataFrame(list(itertools.chain.from_iterable(text_id_list)), columns=['text_id'])

    cv_df = pd.concat([text_id_df, cv_mean_df], axis=1).sort_values('text_id')
    
    return cv_df

# NEED TO LOOK OVER AGAIN
def combine_result(df_list, method=None):
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    text_id_df = df_list[0]['text_id']
    np_pred_list = [df[target_cols].values for df in df_list]
    
    if method == 'mean':
        pred_mean = np.mean(np_pred_list, axis=0)
        pred_mean_df = pd.DataFrame(pred_mean, columns=target_cols)
        result = pd.concat([text_id_df, pred_mean_df], axis=1)
    
    if isinstance(method, list):
        if len(df_list) != len(method):
            raise Exception('Weight len or df_list is not equal')
        else:
            pred_weighted_mean = np.average(np_pred_list, axis=0, weights=method)
            pred_weighted_mean_df = pd.DataFrame(pred_weighted_mean, columns=target_cols)
            result = pd.concat([text_id_df, pred_weighted_mean_df], axis=1)
    
    return result

In [ ]:
# debertav3b_s42_path = get_model_path_dict('../input/fb3-deberta-v3-base/exp43s42', [0, 1, 2, 3])
# debertav3b_s12_path = get_model_path_dict('../input/fb3-deberta-v3-base/exp43s12', [0, 1, 2, 3])
# debertav3b_s0_path = get_model_path_dict('../input/fb3-deberta-v3-base/exp43s0', [0, 1, 2, 3])

# debertav3b_s42_pl1_path = get_model_path_dict('/content/drive/MyDrive/fb3/results/exp58s42', [0, 1, 2, 3])
debertav3b_s42_pl2_path = get_model_path_dict('/content/drive/MyDrive/fb3/results/exp64s42', [0, 1, 2, 3])

# debertav3b_oor_s42_path = get_model_path_dict('../input/fb3-deberta-v3-base/oor0', [0, 1, 2, 3, 4])

# debertav3l_s42_path = get_model_path_dict('../input/fb3-deberta-v3-large/exp50s42', [0, 1, 2, 3])
# debertav3l_s12_path = get_model_path_dict('../input/fb3-deberta-v3-large/exp50s12', [0, 1, 2, 3])

# debertav3l_s12_pl1_path = get_model_path_dict('/content/drive/MyDrive/fb3/results/exp62s12', [0, 1, 2, 3])

# bigbirdb_s42_path = get_model_path_dict('../input/fb3-bigbird-roberta-base/exp49s42', [0, 1, 2, 3])
# bigbirdb_s12_path = get_model_path_dict('../input/fb3-bigbird-roberta-base/exp49s12', [0, 1, 2, 3])
# bigbirdb_s0_path = get_model_path_dict('../input/fb3-bigbird-roberta-base/exp49s0', [0, 1, 2, 3])

# bigbirdb_s42_pl1_path = get_model_path_dict('/content/drive/MyDrive/fb3/results/exp61s42', [0, 1, 2, 3])

# robertal_s42_path = get_model_path_dict('../input/fb3-roberta-large/exp54s42', [0, 1, 2, 3])
# robertal_s12_path = get_model_path_dict('../input/fb3-roberta-large/exp54s12', [0, 1, 2, 3])
# robertal_s0_path = get_model_path_dict('/content/drive/MyDrive/fb3/results/exp54s0', [0, 1, 2, 3])

# robertal_s42_pl1_path = get_model_path_dict('/content/drive/MyDrive/fb3/results/exp63s42', [0, 1, 2, 3])
# robertal_s12_pl1_path = get_model_path_dict('/content/drive/MyDrive/fb3/results/exp63s12', [0, 1, 2, 3])

# longformerb_s42_path = get_model_path_dict('../input/fb3-longformer-base', [0, 1, 2, 3])

# longformerl_s42_path = get_model_path_dict('../input/fb3-longformer-large/exp57s42', [0, 1, 2, 3])
# longformerl_s12_path = get_model_path_dict('../input/fb3-longformer-large/exp57s12', [0, 1, 2, 3])

# exp_num = 13
# exp_dict = get_model_path_dict(f'../input/deberta-v3-base-upload/exp{exp_num}/exp{exp_num}', [0])
# from_nb_dict = get_model_path_dict('../input/offline-model-fb3', [0])

In [ ]:
interact = False
tuning = False

In [ ]:
target_df = train_df[:].copy()

In [ ]:
if not interact:
#     debertav3b_s42_oof_df = inference_cv(debertav3b_s42_path, target_df)
#     debertav3b_s12_oof_df = inference_cv(debertav3b_s12_path, target_df)
#     debertav3b_s0_oof_df = inference_cv(debertav3b_s0_path, target_df)

    # debertav3b_s42_pl1_oof_df = inference_cv(debertav3b_s42_pl1_path, target_df)
    debertav3b_s42_pl2_oof_df = inference_cv(debertav3b_s42_pl2_path, target_df)

#     debertav3b_oor_s42_oof_df = inference_cv(debertav3b_oor_s42_path, target_df)
    
#     debertav3l_s42_oof_df = inference_cv(debertav3l_s42_path, target_df)
#     debertav3l_s12_oof_df = inference_cv(debertav3l_s12_path, target_df)

    # debertav3l_s12_pl1_oof_df = inference_cv(debertav3l_s12_pl1_path, target_df)
    
#     bigbirdb_s42_oof_df = inference_cv(bigbirdb_s42_path, target_df)
#     bigbirdb_s12_oof_df = inference_cv(bigbirdb_s12_path, target_df)

    # bigbirdb_s42_pl1_oof_df = inference_cv(bigbirdb_s42_pl1_path, target_df)

#     robertal_s42_oof_df = inference_cv(robertal_s42_path, target_df)
#     robertal_s12_oof_df = inference_cv(robertal_s12_path, target_df)
    # robertal_s0_oof_df = inference_cv(robertal_s0_path, target_df)

    # robertal_s42_pl1_oof_df = inference_cv(robertal_s42_pl1_path, target_df)
    # robertal_s12_pl1_oof_df = inference_cv(robertal_s12_pl1_path, target_df)
    
#     longformerb_s42_oof_df = inference_cv(longformerb_s42_path, target_df)
    
#     longformerl_s42_oof_df = inference_cv(longformerl_s42_path, target_df)
#     longformerl_s12_oof_df = inference_cv(longformerl_s12_path, target_df)

Using "/content/drive/MyDrive/fb3/results/exp64s42/deberta-v3-base_config.pt"
Loaded "/content/drive/MyDrive/fb3/results/exp64s42/best-epoch-fold0.pt"


  0%|          | 0/978 [00:00<?, ?it/s]

Loaded "/content/drive/MyDrive/fb3/results/exp64s42/best-epoch-fold1.pt"


  0%|          | 0/977 [00:00<?, ?it/s]

Loaded "/content/drive/MyDrive/fb3/results/exp64s42/best-epoch-fold2.pt"


  0%|          | 0/978 [00:00<?, ?it/s]

Loaded "/content/drive/MyDrive/fb3/results/exp64s42/best-epoch-fold3.pt"


  0%|          | 0/978 [00:00<?, ?it/s]

In [ ]:
if not interact:
#     debertav3b_s42_oof_df.to_csv('deberta3b_s42_oof_df.csv', index=False)
#     debertav3b_s12_oof_df.to_csv('deberta3b_s12_oof_df.csv', index=False)
#     debertav3b_s0_oof_df.to_csv('deberta3b_s0_oof_df.csv', index=False)

    # debertav3b_s42_pl1_oof_df.to_csv('/content/drive/MyDrive/fb3/results/exp58s42/debertav3b_s42_pl1_oof_df.csv', index=False)
    debertav3b_s42_pl2_oof_df.to_csv('/content/drive/MyDrive/fb3/results/exp64s42/debertav3b_s42_pl2_oof_df.csv', index=False)
    
#     debertav3b_oor_s42_oof_df.to_csv('debertav3b_oor_s42_oof_df.csv', index=False)
    
#     debertav3l_s42_oof_df.to_csv('deberta3l_s42_oof_df.csv', index=False)
#     debertav3l_s12_oof_df.to_csv('deberta3l_s12_oof_df.csv', index=False)

    # debertav3l_s12_pl1_oof_df.to_csv('/content/drive/MyDrive/fb3/results/exp62s12/debertav3l_s12_pl1_oof_df.csv', index=False)

#     bigbirdb_s42_oof_df.to_csv('bigbirdb_s42_oof_df.csv', index=False)
#     bigbirdb_s12_oof_df.to_csv('bigbirdb_s12_oof_df.csv', index=False)

    # bigbirdb_s42_pl1_oof_df.to_csv('/content/drive/MyDrive/fb3/results/exp61s42/bigbirdb_s42_pl1_oof_df.csv', index=False)

#     robertal_s42_oof_df.to_csv('robertal_s42_oof_df.csv', index=False)
#     robertal_s12_oof_df.to_csv('robertal_s12_oof_df.csv', index=False)
    # robertal_s0_oof_df.to_csv('/content/drive/MyDrive/fb3/results/exp54s0/robertal_s0_oof_df.csv', index=False)

    # robertal_s42_pl1_oof_df.to_csv('/content/drive/MyDrive/fb3/results/exp63s42/robertal_s42_pl1_oof_df.csv', index=False)
    # robertal_s12_pl1_oof_df.to_csv('/content/drive/MyDrive/fb3/results/exp63s12/robertal_s12_pl1_oof_df.csv', index=False)
    
#     longformerb_s42_oof_df.to_csv('longformerb_s42_oof_df.csv', index=False)
    
#     longformerl_s42_oof_df.to_csv('longformerl_s42_oof_df.csv', index=False)
#     longformerl_s12_oof_df.to_csv('longformerl_s12_oof_df.csv', index=False)

# Tuning ensemble weight

In [ ]:
if interact:
    debertav3b_s42_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/deberta3b_s42_oof_df.csv')
    debertav3b_s12_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/deberta3b_s12_oof_df.csv')
    debertav3b_s0_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/deberta3b_s0_oof_df.csv')
    
    debertav3b_oor_s42_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/debertav3b_oor_s42_oof_df.csv')
    
    debertav3l_s42_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/deberta3l_s42_oof_df.csv')
    debertav3l_s12_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/deberta3l_s12_oof_df.csv')

    bigbirdb_s42_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/bigbirdb_s42_oof_df.csv')
    bigbirdb_s12_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/bigbirdb_s12_oof_df.csv')
    bigbirdb_s0_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/bigbirdb_s0_oof_df.csv')
    
    robertal_s42_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/robertal_s42_oof_df.csv')
    robertal_s12_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/robertal_s12_oof_df.csv')
    
    longformerb_s42_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/longformerb_s42_oof_df.csv')
    
    longformerl_s42_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/longformerl_s42_oof_df.csv')
    longformerl_s12_oof_df = pd.read_csv('/kaggle/input/fb3-oof-df/longformerl_s12_oof_df.csv')

In [ ]:
if interact:
    pred_df = combine_result(
        [
            debertav3b_s42_oof_df,
            debertav3b_s12_oof_df,
            debertav3b_s0_oof_df,
            bigbirdb_s42_oof_df,
#             bigbirdb_s12_oof_df,
            debertav3l_s42_oof_df,
            debertav3l_s12_oof_df,
            robertal_s42_oof_df,
            robertal_s12_oof_df,
            longformerl_s42_oof_df,
            longformerl_s12_oof_df,
        ],
        method='mean'
    )
    
    oof_cv_df = get_oof_df(target_df, pred_df)
    score, score_cols = calculate_oof_cv(oof_cv_df)
    
    display(pd.DataFrame(
        np.array([score, *score_cols.numpy().tolist()]).reshape(1, -1),
        columns=['mcrmse', 'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    ))

,mcrmse,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0.4467,0.476814,0.439759,0.409522,0.4484,0.466798,0.438906


In [ ]:
def objective(trial):
    
    model_list = [
        debertav3b_s42_oof_df,
        debertav3b_s12_oof_df,
        debertav3b_s0_oof_df,
        bigbirdb_s42_oof_df,
#         bigbirdb_s12_oof_df,
        debertav3l_s42_oof_df,
        debertav3l_s12_oof_df,
        robertal_s42_oof_df,
        robertal_s12_oof_df,
        longformerl_s42_oof_df,
        longformerl_s12_oof_df,
    ]
    
    num = len(model_list)
    weights = [0.0] * num
    
    for m_w in range(num):
        weights[m_w] = trial.suggest_float(f'w{m_w}', 0, 1)

    pred_df = combine_result(model_list, method=weights)
    
    oof_cv_df = get_oof_df(target_df, pred_df)
    score, score_cols = calculate_oof_cv(oof_cv_df)
    
    
    return score

In [ ]:
if interact and tuning:
    study = optuna.create_study(direction='minimize')
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study.optimize(objective, n_trials=1000)

[I 2022-11-23 19:35:48,305] A new study created in memory with name: no-name-af89546c-09ac-4da4-98fd-b30e63ead66f
[I 2022-11-23 19:35:48,336] Trial 0 finished with value: 0.4460507408892234 and parameters: {'w0': 0.1423079193414426, 'w1': 0.1465350157477413, 'w2': 0.6433051929668461, 'w3': 0.2353874720714214, 'w4': 0.4497486315352601, 'w5': 0.44677000779331333, 'w6': 0.9220634719007111, 'w7': 0.08891257628593552, 'w8': 0.4740728620676843, 'w9': 0.4460182855623168}. Best is trial 0 with value: 0.4460507408892234.
[I 2022-11-23 19:35:48,356] Trial 1 finished with value: 0.44576477833591294 and parameters: {'w0': 0.18143510058811796, 'w1': 0.2772284593500396, 'w2': 0.4521179612842702, 'w3': 0.48656894801729467, 'w4': 0.96038779272741, 'w5': 0.42846600408431856, 'w6': 0.631547329971474, 'w7': 0.17908885536097974, 'w8': 0.16845740274430343, 'w9': 0.19316915066478524}. Best is trial 1 with value: 0.44576477833591294.
[I 2022-11-23 19:35:48,376] Trial 2 finished with value: 0.4458097817300051

In [ ]:
if interact and tuning:
    print('best_trial:', study.best_trial.number)
    print()
    print('best_params:', study.best_params)
    print('best_values:', study.best_value)
    print()
    print(list(study.best_params.values()))
    print(study.best_value)

best_trial: FrozenTrial(number=1530, values=[0.4455866360081983], datetime_start=datetime.datetime(2022, 11, 23, 19, 38, 55, 977260), datetime_complete=datetime.datetime(2022, 11, 23, 19, 38, 56, 143403), params={'w0': 0.26062320168957476, 'w1': 0.2250910878837649, 'w2': 0.5790525780912226, 'w3': 0.25600821924289907, 'w4': 0.9367605099982548, 'w5': 0.6051110192021988, 'w6': 0.32220885446090913, 'w7': 0.5106515117072621, 'w8': 0.3544418096714307, 'w9': 0.39511336697395155}, distributions={'w0': UniformDistribution(high=1.0, low=0.0), 'w1': UniformDistribution(high=1.0, low=0.0), 'w2': UniformDistribution(high=1.0, low=0.0), 'w3': UniformDistribution(high=1.0, low=0.0), 'w4': UniformDistribution(high=1.0, low=0.0), 'w5': UniformDistribution(high=1.0, low=0.0), 'w6': UniformDistribution(high=1.0, low=0.0), 'w7': UniformDistribution(high=1.0, low=0.0), 'w8': UniformDistribution(high=1.0, low=0.0), 'w9': UniformDistribution(high=1.0, low=0.0)}, user_attrs={}, system_attrs={}, intermediate_v

In [ ]:
def scipy_weigh_tune(weights, model_value_list, target_value):
    weighted_result = np.average(model_value_list, axis=0, weights=weights)
    score = np.mean(np.sqrt(np.square(np.subtract(target_value, weighted_result)).mean(axis=0)))
    return score

In [ ]:
if interact and tuning:
    from scipy.optimize import minimize
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    model_list = [
#             debertav3b_s42_oof_df,
            debertav3b_s42_pl1_oof_df,
            debertav3b_s12_oof_df,
#             debertav3b_s12_pl1_oof_df,
            debertav3b_s0_oof_df,
#             debertav3s_s42_oof_df,
#             bigbirdb_s42_oof_df,
            bigbirdb_s42_pl1_oof_df,
#             bigbirdb_s12_oof_df,
            debertav3l_s42_oof_df,
            debertav3l_s42_pl1_oof_df,
            debertav3l_s12_oof_df,
#             debertav3l_s12_pl1_oof_df,
#             debertav3l_s0_oof_df,
#             robertal_s42_oof_df,
            robertal_s42_pl1_oof_df,
            robertal_s12_oof_df,
#             robertal_s12_pl1_oof_df,
#             robertal_s0_oof_df,
#             longformerl_s42_oof_df,
            longformerl_s12_oof_df,
    ]
    model_value_list = [model[target_cols].values for model in model_list]

    target_df_opt = target_df.copy()
    target_opt_value = target_df_opt[target_cols].values

    weights_init = [1] * len(model_list)

    optim_res = minimize(scipy_weigh_tune, weights_init, args=(model_value_list, target_opt_value), method='Nelder-Mead', tol=1e-6)
    print('Optimized weighted ensemble score: ', optim_res.fun)
    print('Weight list: ', optim_res.x.tolist())
    print()
    print(optim_res.x.tolist())
    print(optim_res.fun)